In [1]:
import numpy as np
import pandas as pd

In [2]:
def getNAs(df):

    colNames = []
    percentNA = []
    for i in df.columns:
        colNames.append(i)
        numNA = df[i].isna().sum()
        percent = (numNA/len(df))*100
        percentNA.append(percent)
        
    colNames = pd.DataFrame(colNames)
    colNames = colNames.rename(columns={0: "label"})
    percentNA = pd.DataFrame(percentNA)
    percentNA = percentNA.rename(columns={0: "numNA"})
    d = pd.concat([colNames,percentNA], axis = 1).sort_values(by=['numNA'], ascending = False)

        
    return d

In [3]:
def remove_columns(df, threshold_percent):
    '''drop columns by a threshold (percentage of na)'''
    data = getNAs(df)
    column_names = data[data.numNA >= threshold_percent].label
    clean_data = df.drop(column_names, axis = 1)
    
    return clean_data





In [4]:
#convert all categories to lowercase, there is an issue with factor levels of y, n, N, Y for yes/no 
def entry_to_lowercase(df):
    for i in df.columns:
        if (df[i].dtype == "O"):
            df[i] = semi_clean_crop[i].str.lower()
    return df

In [5]:
### identify which are categorical and which are numeric by looking at the factor levels
## create dataframe with variable name, levels, number of categories and data types

def id_data_types(df):
    names = pd.DataFrame()
    category = pd.DataFrame()
    data_type = pd.DataFrame()
    numUni = pd.DataFrame()

    for i in df.columns:
        
        names = names.append({"variable": i},ignore_index = True)
        category = category.append({"category": list(df[i].unique())}, ignore_index = True)
        numUni = numUni.append({"numUnique": len(df[i].unique())}, ignore_index = True)
        data_type = data_type.append({"data_type": df[i].dtype}, ignore_index = True)
    
    view_data = pd.concat([names, category, numUni,data_type],axis =1)
    
    return (view_data)

In [6]:
def replace_na_with_NaN(df):
    for i in df.columns:
        df[i] = df[i].replace('na', np.NaN)
    return df



In [7]:
rhomis_data = pd.read_csv("/Users/anab/Documents/MS_UCDavis/STA208/project/RHoMIS_Full_Data.csv", engine = "python")
rh_indic = pd.read_csv("/Users/anab/Documents/MS_UCDavis/STA208/project/STA_208/data/RHoMIS_Indicators.csv", engine='python')

In [8]:
## final dataset, keep and drop these columns
rhomis_data = rhomis_data[["crop_count", "crop_name_1", "crop_harvest_1", "crop_intercrop_1"]]
rh_indic = rh_indic.drop(['ITERATION','GPS_LAT', 'GPS_LON', 'GPS_ALT', 'Altitude','WorstFoodSecMonth',
       'BestFoodSecMonth','FIES_Score',  'currency_conversion_factor', 'GHGEmissions' ],axis = 1)

In [ ]:
bigData  = pd.concat([rhomis_data,rh_indic ],axis = 1)
bigData = remove_columns(bigData, 50)
bigData.head(3)


bigData = replace_na_with_NaN(bigData)
bigData.dtypes


In [ ]:
a = bigData.select_dtypes('object')
id_data_types(a.select_dtypes('object'))

bigData.crop_count = bigData.crop_count.fillna(-500)
bigData.crop_count = bigData.crop_count.astype(int)
bigData.crop_count = bigData.crop_count.replace(-500, np.nan)